In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random





#from google.colab.patches import cv2_imshow






# import some common detectron2 utilities

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.modeling import build_model
from detectron2.evaluation import COCOEvaluator,PascalVOCDetectionEvaluator
import matplotlib.pyplot as plt
import torch.tensor as tensor
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import inference_on_dataset
import torch
from detectron2.structures.instances import Instances
from detectron2.modeling import build_model
from detectron2.modeling.meta_arch.tracker import Tracker
from detectron2.modeling.meta_arch.soft_tracker import SoftTracker
%matplotlib inline






## Load Weights

In [ ]:
cfg = get_cfg()
#cfg.MODEL.DEVICE='cpu'
cfg.merge_from_file("../configs/COCO-Detection/faster_rcnn_R_50_FPN_3x_Video.yaml")
#cfg.merge_from_file("./detectron2_repo/configs/COCO-Detection/faster_rcnn_R_50_Video.yaml")
cfg.MODEL.ROI_HEADS.NUM_CLASSES=1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4  # set threshold for this model
#cfg.MODEL.WEIGHTS = "KITTI_FPN_FINAL/model_final.pth"
cfg.MODEL.WEIGHTS = "../models_pub/MOT/mot_17.pth"

print(cfg.MODEL)
#arr = {1:'cyclist',2:'car',0:'pedestrian'}
arr = {0:'Pedestrian'}



## inference: Joint detection and tracking
- fix the hyper parameters (setting)
- choose a dataset (train or test of mot17 or mot20)

In [ ]:
import json
import os
import time
from tqdm.notebook import tqdm
colors = [[0,0,128],[0,255,0],[0,0,255],[255,0,0],[0,128,128],[128,0,128],[128,128,0],[255,255,0],[0,255,255],[255,255,0],[128,0,0],[0,128,0]
         ,[0,128,255],[0,255,128],[255,0,128],[128,255,0],[255,128,0],[128,255,255],[128,0,255],[128,128,128],[128,255,128]]

dirC = '../datasets/MOT/MOT17/train/'
names = []


settings = [
   
    
    
 settings = [
   
    dict(props=50, #number of proposals to use by rpn
         st=1.05, #acceptance distance percentage for soft tracker
         sup_fp = False, # fp suppression based on Intersection over Union for new detections
         alpha = 0.6, # the percentage of the new embedding in track embedding update (emb = alpha * emb(t) +(1-alpha) emb(t-1)) 
         fp_thresh=0.95, # iou threshold above which the new detection is considered a fp
         T=True, #use past tracks as proposals
         D='cosine', # distance metric for embeddings
         Re=True, #use the embedding head 
         A=True, # use appearance information
         H=False, # use HOG features as appearance descriptors
         K=True, # use kalman for motion prediction
         E=False, #use raw FPN features as appearance descriptors
         measurement=0.001, #measruement noise for the kalman filter
         process=1, #process noise for the kalman filter
         hog_cells=4, #number of cells used to calculate hof features
         dist_thresh=1.5, # the normalization factor for the appearance distance
         track_life=7, #frames for which a track is kept in memory without an update
         track_vis=2, #frames for which a track is displayed without an update
        ),
    ]   

  
    
    
]
train_folders_17 = ['MOT17-02','MOT17-04','MOT17-05','MOT17-09','MOT17-10','MOT17-11','MOT17-13']
test_folders_17 = ['MOT17-01','MOT17-03','MOT17-06','MOT17-07','MOT17-08','MOT17-12','MOT17-14']

train_folder_20 = ["MOT20-01","MOT20-02","MOT20-03","MOT20-05"]
test_folder_20 = ["MOT20-04","MOT20-06","MOT20-07","MOT20-08"]

mot15_folders = ['MOT17-02','MOT17-04','MOT17-05','MOT17-09','MOT17-10','MOT17-11','MOT17-13']



for setting in settings:
    
    setting_id = setting_id + 1
    if(not os.path.exists("../results")):
        os.mkdir('../results')
        os.mkdir('../results/MOT')
    else:
        if(not os.path.exists("../results/MOT")):
            os.mkdir('../results/MOT')
    
    output_path = '../results/MOT/MOT20_%s'%(str(setting_id))
    
    exp_name = output_path
    total_elapsed=0
    if(not os.path.exists(exp_name)):
      os.mkdir(exp_name)
      
    for folder_name in test_folders_17:
    
    #for folder_name in os.listdir(dirC):
      #out_tracking = cv2.VideoWriter('joint_%s.avi'%folder_name,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (1242,375))
      
      #if(not folder_name in ['0000','0007','0011']):
        #continue
        
      dump_image = cv2.imread(dirC+folder_name+'/img1/000001.jpg')
      #out_tracking = cv2.VideoWriter('mot_paper.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30, dump_image.shape[1::-1])
      predictor = DefaultPredictor(cfg)
      #prop_limit=60
      predictor.model.tracker = SoftTracker()
      print(folder_name)
      predictor.model.tracking_proposals = setting['T']
      predictor.model.tracker.track_life = setting['track_life']
      predictor.model.tracker.track_visibility = setting['track_vis']
      predictor.model.tracker.use_appearance = setting['A']
      predictor.model.tracker.use_kalman = setting['K']
      predictor.model.tracker.hot = setting['H']
      predictor.model.tracker.use_color = False
      predictor.model.tracker.embed = setting['E']
      predictor.model.tracker.reid = setting['Re']
      predictor.model.tracker.hog = setting['H']
      predictor.model.tracker.measurement_noise=setting['measurement']
      predictor.model.tracker.process_noise = setting['process']
      predictor.model.enable_clustering=False
      predictor.model.tracker.hog_num_cells = setting['hog_cells']
      predictor.model.tracker.dist_thresh = setting['dist_thresh']
      predictor.model.use_reid = setting['Re']
      predictor.model.tracker.soft_thresh = setting['st']
      predictor.model.tracker.suppress_fp = setting['sup_fp']
      predictor.model.tracker.fp_thresh = setting['fp_thresh']
      predictor.model.tracker.embed_alpha = setting['alpha']
      
      max_distance = 0.2
      
      
      
        
      output_file = open('%s/%s.txt'%(exp_name,folder_name),'w')
      print('%s/%s.txt'%(exp_name,folder_name))
      
      start = time.time()
      frame_counter = 0
      prev_path = 0
      predictor.model.prev_path = 0
      frames = {}
      for photo_name in sorted(os.listdir(dirC+folder_name+'/img1/')):
        img_path = dirC+folder_name+'/img1/'+photo_name
        
        frames[frame_counter] = {}
        img = cv2.imread(img_path)
        inp = {}
        inp['width'] = img.shape[1]
        inp['height'] = img.shape[0]



        inp['file_name'] =  photo_name
        inp['image_id'] = photo_name
        
        predictor.model.photo_name = img_path
        
        outputs = predictor(img,setting['props'])
        
       
       
        for i in outputs:

            if(i.pred_class in arr):
              
              output_file.write("%d,%d,%d,%d,%d,%d,%f,-1,-1,-1\n"
                                %(frame_counter,i.track_id,i.xmin,i.ymin,i.xmax-i.xmin,i.ymax-i.ymin
                                  ,i.conf))
      
        frame_counter +=1
        predictor.model.prev_path = img_path

      end = time.time()
      elapsed = end-start

      avg = frame_counter/elapsed
      print('avg time is' ,avg)
      print('elapsed : ',elapsed)
      output_file.close()
      total_elapsed += elapsed
    print('total elapsed ', total_elapsed)
    